[collab](https://colab.research.google.com/github/PanyshevAlex/TSPArt/blob/main/TSP_Art_v2.ipynb)

In [ ]:
import cv2
import glob
import os
import re
import sys
from svglib.svglib import svg2rlg
from reportlab.graphics import renderPM
from xml.dom import minidom

In [ ]:
!git clone https://github.com/thegrandpoobah/voronoi

In [ ]:
!make -C voronoi

In [ ]:
!git clone https://github.com/PanyshevAlex/traveling-salesman

In [ ]:
!make -C traveling-salesman

In [ ]:
!git clone https://github.com/beckysag/traveling-salesman traveling_salesman_2

In [ ]:
!make -C traveling_salesman_2

In [ ]:
# Разбиваем видео на кадры
def video_to_frames(path):
    videoCapture = cv2.VideoCapture()
    videoCapture.open(path)
    fps = videoCapture.get(cv2.CAP_PROP_FPS) 
    frames = videoCapture.get(cv2.CAP_PROP_FRAME_COUNT)
    print("fps=", int(fps), "frames=", int(frames))
    count = 0
    for i in range(int(frames)-1):
        ret, frame = videoCapture.read()
        if type(frame) != type(None):
            cv2.imwrite("source/frames/%d.png"%(i), frame)
            count += 1
    return count

# создание файла для tsp solver формата: index x_coord y_coord 
def create_tsp(path, dest):
    doc = minidom.parse(path)
    circe_x = [path.getAttribute('cx') for path
                    in doc.getElementsByTagName('circle')]
    circe_y = [path.getAttribute('cy') for path
                    in doc.getElementsByTagName('circle')]
    f = open(dest, "w")
    for i in range(len(circe_x)):
        f.write(str(i) + " " + str(int(float(circe_x[i]))) + " " + str(int(float(circe_y[i]))) + "\n")

# создание svg файла решения по точечному изображению и решению tsp
def tsppng(frame, tappath, dest):
    f = open(tappath)
    tsppath = f.readlines()
    ff = open(frame)
    tspcoord = ff.readlines()
    tspcoord = [x.split() for x in tspcoord]
    svg_path = '''<path d="M{} {} '''.format(tspcoord[int(tsppath[1])][1], tspcoord[int(tsppath[1])][2])
    print(svg_path)
    for i in range(2, len(tsppath)):
        svg_path += "L {} {} ".format(tspcoord[int(tsppath[i])][1], tspcoord[int(tsppath[i])][2])
    #svg_path += '''L {} {} '''.format(tspcoord[int(tsppath[1])][1], tspcoord[int(tsppath[1])][2])
    svg_path += '''" stroke="black" stroke-width="3" fill="none"/> \n </svg>'''
    f = open(dest, "a")
    f.write('''<?xml version="1.0" ?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN" "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<svg width="1080" height="934" version="1.1" xmlns="http://www.w3.org/2000/svg">\n''')
    f.write(svg_path)
    f.close()
    ff.close()



In [ ]:
# Полная обработка видео
def solve_video(path, N):
    frames = video_to_frames(path) # разбиваем ка кадры
    
    comm = "./voronoi/voronoi_stippler -I source/frames/{}.png -O source/tippled_frames/{}.svg -s {} -t 0.4 -z 0.5" # команда для запуска voronoi-stippler
    # для каждого кадра создаем точечное изображение
    for i in range(frames):
        os.system(comm.format(i,i, N))
    
    # конвертируем точечные изображения в png 
    for i in range(1,frames):
        filename = "source/tippled_frames/{}.svg".format(i)
        drawing = svg2rlg(filename)
        renderPM.drawToFile(drawing, "source/tippled_frames_png/{}.png".format(i), fmt="PNG")

    # создаем промежуточное точечное видео 
    img_array = []

    for i in range(frames):
        filename = "source/tippled_frames_png/{}.png".format(i)
        img = cv2.imread(filename)
        if type(img) != type(None):
            height, width, layers = img.shape
            size = (width, height)
            img_array.append(img)

    video_out = cv2.VideoWriter('stippled.mp4', cv2.VideoWriter_fourcc(*'MP4V'), 24, size)

    for i in range(len(img_array)):
        video_out.write(img_array[i])
    video_out.release()

    # подготавливаем данные для tsp solver 
    for i in range(frames):
        create_tsp("source/tippled_frames/{}.svg".format(i), "source/tsp_frames/{}.tsp".format(i))
    
    # решаем tsp через nearest neighbour + 2-opt
    os.system("./traveling-salesman/tsp source/tsp_frames/{}.tsp source/tsp_path/{}.tsp.tour".format(0, 0))
    for i in range(1, frames):
        os.system("./traveling-salesman/tsp source/tsp_frames/{}.tsp source/tsp_path/{}.tsp.tour source/tsp_path/{}.tsp.tour source/tsp_frames/{}.tsp".format(i, i, i-1, i-1))


    # создаем svg решения
    for i in range(frames):
        tsppng("source/tsp_frames/{}.tsp".format(i), "source/tsp_path/{}.tsp.tour".format(i), "source/final_frames/{}.svg".format(i))

    # конвертируем svg в png
    for i in range(frames):
        filename = "source/final_frames/{}.svg".format(i)
        drawing = svg2rlg(filename)
        renderPM.drawToFile(drawing, "source/final_frames_png/{}.png".format(i), fmt="PNG")

    # собираем результат в видео
    img_array = []

    for i in range(1,frames):
        filename = "source/final_frames_png/{}.png".format(i)
        img = cv2.imread(filename)
        if type(img) != type(None):
            height, width, layers = img.shape
            size = (width, height)
            img_array.append(img)

    video_out = cv2.VideoWriter('stippled_tspart.mp4', cv2.VideoWriter_fourcc(*'MP4V'), 24, size)

    for i in range(len(img_array)):
        video_out.write(img_array[i])
    video_out.release()

    
    

In [ ]:
solve_video("source/test1.mp4", 1500)

### Сравнение двух методов: nearest neighbour + 2-opt и  Christofides’ algorithm + 2-opt

In [ ]:
import time
from matplotlib import pyplot as plt
import numpy as np

In [ ]:

time_nearest = []
time_christofides = []
f = open("source/tsp_frames/0.tsp")
tsp_inpit = f.readlines()
for i in range(30): 
        ff = open("source/tsp_frames/temp.tsp", "a")
        for j in range(50):
            ff.write(tsp_inpit[50*i+j])
        ff.close()
        stime = time.time()
        os.system("./traveling-salesman/tsp source/tsp_frames/temp.tsp source/tsp_path/firsttime.tsp.tour")
        etime = time.time()
        time_nearest.append(etime-stime)

        stime = time.time()
        os.system("./traveling_salesman_2/tsp source/tsp_frames/temp.tsp")
        etime = time.time()
        time_christofides.append(etime-stime)
        print(time_nearest[i], time_christofides[i])

In [ ]:
N = np.linspace(50, 1500, 30)
plt.plot(N, time_nearest, label="nearest neighbour")
plt.plot(N, time_christofides, label="Christofides algorithm")
plt.xlabel("N")
plt.xlabel("Time")
plt.grid()
plt.legend()
plt.show()




In [ ]:
os.system("./traveling-salesman/tsp source/tsp_frames/{}.tsp source/tsp_path/{}.tsp.tour source/tsp_path/{}.tsp.tour".format(1, 1, 0))
#os.system("./traveling-salesman/tsp source/tsp_frames/{}.tsp source/tsp_path/{}.tsp.tour".format(1, 1))

In [ ]:
tsppng("source/tsp_frames/{}.tsp".format(1), "source/tsp_path/{}.tsp.tour".format(1), "source/final_frames/{}.svg".format(1))

In [ ]:
!git clone https://github.com/ReScience-Archives/Rougier-2017

In [ ]:
os.system("python3 Rougier-2017/code/stippler.py --n_iter 10 --n_point 1000 --display source/frames/0.png")


In [ ]:
%matplotlib inline
from Rougier.code.stippler import initialization, normalize
from Rougier.code.voronoi import centroids
import tqdm
import os.path
import scipy.misc
import scipy.ndimage
import numpy as np
import cv2
import argparse
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import time

In [ ]:
def make_tsp_by_points(path, points):
    f = open(path, "w")
    points = sorted(points, key=lambda x: x[0]+x[1])
    out = []
    temp = np.copy(points)
    out.append(temp[0])
    temp = np.delete(temp, 0, 0)
    current_num = len(temp) 
    closest_index = 0
    while current_num != 0:
        closest = 99999
        for i in range(current_num):
            current_dist = (temp[i][0]-out[-1][0])**2+(temp[i][1]-out[-1][1])**2
            if current_dist < closest:
                closest = current_dist
                closest_index = i
        closest_index = 0
        out.append(temp[closest_index])
        temp = np.delete(temp, closest_index, 0)
        current_num -= 1


    #points = sorted(points, key=lambda x: x[0]+x[1])
    for i in range(len(out)):
        f.write("{} {} {}\n".format(i, int(out[i][0]), int(out[i][1])))
    

In [ ]:
frames = video_to_frames("source/test5.mp4")

In [ ]:
n_point = 1500
threshold = 255
n_iter = 10
figsize = 6
pointsize = (1.0, 1.0)

filename = "source/frames/0.png"
density = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
zoom = (n_point * 500) / (density.shape[0]*density.shape[1])
zoom = int(round(np.sqrt(zoom)))
density = scipy.ndimage.zoom(density, zoom, order=0)

density = np.minimum(density, threshold)

density = 1.0 - normalize(density)
density = density[::-1, :]
density_P = density.cumsum(axis=1)
density_Q = density_P.cumsum(axis=1)

dirname = os.path.dirname(filename)
basename = (os.path.basename(filename).split('.'))[0]
pdf_filename = os.path.join(dirname, basename + "-stipple.pdf")
png_filename = os.path.join(dirname, basename + "-stipple.png")
dat_filename = os.path.join(dirname, basename + "-stipple.npy")

points = initialization(n_point, density)

xmin, xmax = 0, density.shape[1]
ymin, ymax = 0, density.shape[0]
bbox = np.array([xmin, xmax, ymin, ymax])
ratio = (xmax-xmin)/(ymax-ymin)

for i in tqdm.trange(30):
    regions, points = centroids(points, density, density_P, density_Q)

fig = plt.figure(figsize=(figsize, figsize/ratio), facecolor="white")

ax = fig.add_axes([0, 0, 1, 1], frameon=False)
ax.set_xlim([xmin, xmax])
ax.set_xticks([])
ax.set_ylim([ymin, ymax])
ax.set_yticks([])
scatter = ax.scatter(points[:, 0], points[:, 1], s=1, facecolor="k", edgecolor="None")
Pi = points.astype(int)
X = np.maximum(np.minimum(Pi[:, 0], density.shape[1]-1), 0)
Y = np.maximum(np.minimum(Pi[:, 1], density.shape[0]-1), 0)
sizes = (pointsize[0] + (pointsize[1]-pointsize[0])*density[Y, X])
scatter.set_offsets(points)
scatter.set_sizes(sizes)
#np.save(dat_filename, points)
#plt.savefig(pdf_filename)
plt.savefig(png_filename)
plt.close(fig)
make_tsp_by_points("source/tsp_frames/0.tsp", points)

In [ ]:
def referring_stippler(filename, points):
    density = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
    zoom = (n_point * 500) / (density.shape[0]*density.shape[1])
    zoom = int(round(np.sqrt(zoom)))
    density = scipy.ndimage.zoom(density, zoom, order=0)

    density = np.minimum(density, threshold)

    density = 1.0 - normalize(density)
    density = density[::-1, :]
    density_P = density.cumsum(axis=1)
    density_Q = density_P.cumsum(axis=1)

    dirname = os.path.dirname(filename)
    basename = (os.path.basename(filename).split('.'))[0]
    #pdf_filename = os.path.join(dirname, basename + "-stipple.pdf")
    png_filename = os.path.join(dirname, basename + "-stipple.png")
    #dat_filename = os.path.join(dirname, basename + "-stipple.npy")

    #points = initialization(n_point, density)

    xmin, xmax = 0, density.shape[1]
    ymin, ymax = 0, density.shape[0]
    bbox = np.array([xmin, xmax, ymin, ymax])
    ratio = (xmax-xmin)/(ymax-ymin)

    for i in range(n_iter):
        regions, points = centroids(points, density, density_P, density_Q)
    fig = plt.figure(figsize=(figsize, figsize/ratio), facecolor="white")
    ax = fig.add_axes([0, 0, 1, 1], frameon=False)
    ax.set_xlim([xmin, xmax])
    ax.set_xticks([])
    ax.set_ylim([ymin, ymax])
    ax.set_yticks([])
    scatter = ax.scatter(points[:, 0], points[:, 1], s=1, facecolor="k", edgecolor="None")
    Pi = points.astype(int)
    X = np.maximum(np.minimum(Pi[:, 0], density.shape[1]-1), 0)
    Y = np.maximum(np.minimum(Pi[:, 1], density.shape[0]-1), 0)
    sizes = (pointsize[0] + (pointsize[1]-pointsize[0])*density[Y, X])
    scatter.set_offsets(points)
    scatter.set_sizes(sizes)
    #np.save(dat_filename, points)
    #plt.savefig(pdf_filename)
    plt.savefig(png_filename)
    time.sleep(0.1)
    plt.close(fig)
    return points

In [ ]:
for i in tqdm.trange(1, 149):
    points = referring_stippler("source/frames/{}.png".format(i), points)
    make_tsp_by_points("source/tsp_frames/{}.tsp".format(i), points)
    

In [ ]:

frames = 117
os.system("./traveling-salesman/tsp source/tsp_frames/{}.tsp source/tsp_path/{}.tsp.tour".format(0, 0))
for i in range(1, frames):
    os.system("./traveling-salesman/tsp source/tsp_frames/{}.tsp source/tsp_path/{}.tsp.tour source/tsp_path/{}.tsp.tour source/tsp_frames/{}.tsp".format(i, i, i-1, i-1))


# создаем svg решения
for i in range(frames):
    tsppng("source/tsp_frames/{}.tsp".format(i), "source/tsp_path/{}.tsp.tour".format(i), "source/final_frames/{}.svg".format(i))

# конвертируем svg в png
for i in range(frames):
    filename = "source/final_frames/{}.svg".format(i)
    drawing = svg2rlg(filename)
    renderPM.drawToFile(drawing, "source/final_frames_png/{}.png".format(i), fmt="PNG")

# собираем результат в видео
img_array = []

for i in range(frames):
    filename = "source/final_frames_png/{}.png".format(i)
    img = cv2.imread(filename)
    if type(img) != type(None):
        height, width, layers = img.shape
        size = (width, height)
        img_array.append(img)

video_out = cv2.VideoWriter('stippled_tspart.mp4', cv2.VideoWriter_fourcc(*'MP4V'), 24, size)

for i in range(len(img_array)):
    video_out.write(img_array[i])
video_out.release()

In [ ]:
os.system("./traveling-salesman/tsp source/tsp_frames/{}.tsp source/tsp_path/{}.tsp.tour source/tsp_path/{}.tsp.tour source/tsp_frames/{}.tsp".format(1, 1, 0, 0))

In [ ]:
tsppng("source/tsp_frames/{}.tsp".format(1), "source/tsp_path/{}.tsp.tour".format(1), "source/tsp_path/{}.svg".format(1))

In [ ]:
!make -C traveling-salesman

In [ ]:
frames = 138
os.system("./traveling-salesman/tsp source/tsp_frames/{}.tsp source/tsp_path/{}.tsp.tour".format(0, 0))
for i in range(1, frames):
    os.system("./traveling-salesman/tsp source/tsp_frames/{}.tsp source/tsp_path/{}.tsp.tour source/tsp_path/{}.tsp.tour source/tsp_frames/{}.tsp".format(i, i, i-1, i-1))


    # создаем svg решения
for i in range(frames):
    tsppng("source/tsp_frames/{}.tsp".format(i), "source/tsp_path/{}.tsp.tour".format(i), "source/final_frames/{}.svg".format(i))

    # конвертируем svg в png
for i in range(frames):
    filename = "source/final_frames/{}.svg".format(i)
    drawing = svg2rlg(filename)
    renderPM.drawToFile(drawing, "source/final_frames_png/{}.png".format(i), fmt="PNG")

    # собираем результат в видео
img_array = []

for i in range(1,frames):
    filename = "source/final_frames_png/{}.png".format(i)
    img = cv2.imread(filename)
    if type(img) != type(None):
        height, width, layers = img.shape
        size = (width, height)
        img_array.append(img)

video_out = cv2.VideoWriter('stippled_tspart.mp4', cv2.VideoWriter_fourcc(*'MP4V'), 24, size)

for i in range(len(img_array)):
    video_out.write(img_array[i])
video_out.release()

In [ ]:
import time

t1 = []
stime = time.time()
os.system("./traveling-salesman/tsp source/tsp_frames/{}.tsp source/tsp_path/{}.tsp.tour".format(0, 0))
etime = time.time()
t1.append(etime - stime)
for i in range(1, 149):
    os.system("./traveling-salesman/tsp source/tsp_frames/{}.tsp source/tsp_path/{}.tsp.tour source/tsp_path/{}.tsp.tour source/tsp_frames/{}.tsp".format(i, i, i-1, i-1))
    etime = time.time()
    t1.append(etime - stime)

t2 = []
stime = time.time()
for i in range(149):
    os.system("./traveling-salesman/tsp source/tsp_frames/{}.tsp source/tsp_path/{}.tsp.tour".format(i, i))
    etime = time.time()
    t1.append(etime - stime)
print(t1)
print(t2)

In [ ]:
N = np.linspace(0, 148, 149)
print(len(N))
print(len(t1[:149]))
print(len(t1[149:298]))
plt.plot(N, t1[:149], label="Using the previous frame")
plt.plot(N, t1[149:298], label="Without using the previous frame")
plt.xlabel("Number of frames")
plt.ylabel("Time")

plt.grid()
plt.legend()
plt.show()
